# Genetic Algorithm

Feature Selection is one of the core concepts in machine learning, which greatly impacts the performance of a model. It refers to the process of choosing the most relevant features in the data to give to our model. The Genetic Algorithm (GA), a heuristic approach, has a great advantage for efficient feature selection by providing close to optimum feature subset within a reasonable amount of time. It is a stochastic method for function optimization inspired by evolution by natural selection, where organisms' genes evolve over generations to better adapt to the environment.

Genetic algorithms operate on a population of chromosomes to produce better estimates. In each generation, individuals are selected based on their fitness scores in the problem domain. They are then recombined to create a new population using operators borrowed from natural genetics like cross-over and mutation. 

A general overview of the steps involved in feature selection with the genetic algorithm is illustrated below:


<img src="Images/genalg.PNG">

In [1]:
# Importing relevant libraries

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from csv import writer


In [4]:
# Function to add list of elements to a row in a given csv file

def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)
        


## Fitness Assignment
For each individual in a population, a fitness value is assigned. The most used method for fitness assignment is the rank-based method where a higher rank is assigned to the individual that performs better. Thus, it dependens on the relative performance rather than the individual score value. In this approach, the performance of each chromosome is first evaluated and the scores are stored in an array. Individuals are then selected based on how high their score is in the array.

To evaluate the performance of each chromosome, we have implemented a function, **evaluate_GA**, with inputs:

 - *chromosome*: the binary string that represents the selected features for that solution

 - *X*: input with all fluxes

 - *y*: true values of the predicted phenotype

 - *clf*: the ML model to be tuned and used for prediction

and an output: *Score* - an array of parameters that include:

 - *clf.best_score_*: R<sup>2</sup> evaluated for the cross-validation folds in the grid search algorithm

 - *best_clf*: the best hyperparameters for the model found using grid search algorithm


The calculated scores for each chromosome are combined to produce arrays representing the entire population in **cal_pop_fitness**

In [ ]:
## Calculate the fitness score for the given chromosome

def evaluate_GA(chromosome, X, y,clf):
    
    Fluxes = np.asarray(X.columns)

    #Get index of non-zero elements in the feature list
    F_names = [Fluxes[idx] for idx, val in enumerate(chromosome) if val != 0]

    #Get corresponding X
    X_flx = X[F_names]

    clf.fit(X_flx,y)

    Param = clf.best_estimator_
    best_clf = Param.steps[1][1]
    Score = [clf.best_score_, best_clf]
    return Score

## Calculate Population Fitness

def cal_pop_fitness(population,X,y,clf):
    fitness = np.empty(population.shape[0])
    best_clfs = [None]*(population.shape[0])
    fIdx = 0
    for chromosome in population:
        score_chrom = evaluate_GA(chromosome, X, y,clf)
        fitness[fIdx] = score_chrom[0]
        best_clfs[fIdx] = score_chrom[1]
        fIdx+=1
    return (fitness,best_clfs)


## Selection of parents and creation of offsprings

After assigning fitness values to each individual, the **select_mating_pool** function is used to choose the individuals to recombine for the next generation. The individuals that adapt the most to the environment are more likely to survive. Therefore, the selection operator selects the individuals according to their fitness level. We also use elitism selection and let the fittest individuals survive directly for the next generation. In this approach, we use the array of fitness scores evaluated in cal_pop_fitness routine.

The **crossover** operator recombines the selected individuals from the mating pool to generate a new population. This operator picks two individuals at once in a round-robin fashion. Each chromosome string is split in half and the left half of the first string is recombined with the right half of the second string to create a new offspring. This process is repeated until we have a new population of size defined by the parameter *offspring_size*.

The crossover operator can generate offsprings that are very similar to the parents, resulting in a new generation with low diversity. The **mutation** operator solves this issue by randomly flipping some bits of the offsprings that are same as the individuals in the given population. 

In [ ]:
## Select parents from the mating pool

def select_mating_pool(population, fitness, num_parents_mating):
    # Initialize parents array
    parents = np.empty((num_parents_mating, population.shape[1]))

    for parent_num in range(num_parents_mating):
        # Obtain the required chromosomes, having the highest fitness values in the population
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = population[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999

    return parents

## Create offsprings by crossover between parents

def crossover(parents, offspring_size):
    # parents: Each row has a parent chromosome
    #offspring_size: index0- no. of offsprings desired. index1- length of chromosome

    #Initialize the offspring array
    offspring = np.empty(offspring_size)

     # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = np.uint8(offspring_size[1]/2)
    add_on = 1
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+add_on)%parents.shape[0]
        if (parent2_idx==0):
            add_on+=1
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

    offspring = check_pop(offspring)
    return offspring

## Create offsprings by mutating parents

def mutation(offspring,parents):

    # Mutation flips four random genes in the offsprings that has low diversity

    for idx in range(offspring.shape[0]):
        # The offspring is mutated until it no longer resembles one of its parents
        while offspring[idx] in parents:
            # Selecting the random genes to be changed
            flipped_bits = 4
            random_idx = np.random.randint(offspring.shape[1], size =flipped_bits)
            for r_id in random_idx:
                # The gene at r_id position in the chromosome is flipped
                offspring[idx, r_id] = 1 - (offspring[idx, r_id]) 
                
    # Final check of the offspring population
    offspring=check_pop(offspring)
    return offspring


We now implement the genetic algorithm, **GA_fs**, where the fitness assignment, selection, crossover, and mutation processes are repeated until the stopping criterion, iterating for n generations, is satisfied. When the criterion is reached, the genetic algorithm outputs the fittest chromosome of the last generation as the best solution. 

In [ ]:
## Rudimentary Check to ensure the randomly generated population doesn't contain an all-zero chromosome

def check_pop(pop_i):
    for i in range(pop_i.shape[0]):
        if (sum(pop_i[i])==0):
            pop_i[i][np.random.randint(pop_i.shape[1])]=1
    return pop_i

## Implementation of Genetic Algorithm

def GA_fs(X,y,clf,Phen):
    num_generations=6
    num_weights = len(X.columns)
    sol_per_pop = 35
    num_parents_mating = 28
    parents_chosen=5


    # Defining the population size.
    pop_size = (sol_per_pop,num_weights)
    # The population will have sol_per_pop chromosomes where each chromosome has num_weights genes.


    #Creating the initial population.
    new_population = np.random.randint(2, size=pop_size)
    new_population[1]=[1]*num_weights
    new_population= check_pop(new_population) #Ensure no solution is all zeros

    # File to save results from each generation
    file = 'gen_fit_'+Phen+'.csv' #Enter the file directory with the file name here
    row_names = ['Generation','Cross-validation score','Best_model_parameters','Selected_flux_variables']
    append_list_as_row(file,row_names)

    for generation in range(num_generations):
        print('Gen',generation)
        # Measuring the fitness of each chromosome in the population.
        (fitness,best_clfs) = cal_pop_fitness(new_population,X,y,clf)
        f_max = np.max(fitness)
        Fluxes = np.asarray(X.columns)
        #Get required X
        max_fitness_idx = np.where(fitness == f_max)
        max_fitness_idx = max_fitness_idx[0][0]
        best_chrom = new_population[max_fitness_idx, :]
        best_clf = best_clfs[max_fitness_idx]
        #Get index of non-zero elements in the feature list
        F_names = [Fluxes[idx] for idx, val in enumerate(best_chrom) if val != 0]
        res = [generation, f_max , best_clf, F_names]
        append_list_as_row(file,res)

        #print(np.max(fitness))
        # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
        parents = select_mating_pool(new_population, fitness,num_parents_mating)
        # Generating next generation using crossover.
        offspring_crossover = crossover(parents, offspring_size=(sol_per_pop-parents_chosen, num_weights))
        # Adding some variations to the offsrping using mutation.
        offspring_mutation = mutation(offspring_crossover,parents)

        # Creating the new population based on the parents and offspring.
        new_population[0:parents_chosen, :] = parents[0:parents_chosen, :]
        new_population[parents_chosen:, :] = offspring_mutation


    (fitness,best_clfs) = cal_pop_fitness(new_population,X,y,clf)
    f_max = np.max(fitness)
    Fluxes = np.asarray(X.columns)
    #Get required X
    max_fitness_idx = np.where(fitness == f_max)
    max_fitness_idx = max_fitness_idx[0][0]
    best_chrom = new_population[max_fitness_idx, :]
    best_clf = best_clfs[max_fitness_idx]
    #Get index of non-zero elements in the feature list
    F_names = [Fluxes[idx] for idx, val in enumerate(best_chrom) if val != 0]

    return (F_names,best_clf,f_max)
